In [ ]:
import numpy as np
from __future__ import division

In [ ]:
import picos as pic
import cvxopt as cvx

In [ ]:
from scipy.special import comb as nchoosek # n Choose k (n ueber k)

# As seen in A.4 jasniwiecz
# Intervalltransformation (Gleichung (4.36))
# p => [p_l, p_u] (p => [p_min, 1]) wird zu p^ => [-1,1] return neue Matrizen(!) mit diesen p^
#
# Q_in ist Liste aus Faktormatrizen für Polynom in p
# Q_out ist Liste aus Faktormatrizen für Polynom in p^
# a = (p_u - p_l)
# b = (p_u + p_l)
def transQ(Q_in, pl=0.01, pu=1):
    a = pu - pl
    b = pu + pl
    m = len(Q_in) # Anzahl der Koeffizienten in Q_in
    n = Q_in[0].shape

    #print Q_in
    Q_out = [np.zeros(Q_in[0].shape)] * m
    #print Q_out
    for j in range(0, m):
        #print j
        #Q_out[j] = Q_in[j]
        for i in range(j, m):
            #print Q_out[j]
            Q_out[j] = Q_out[j] + 2**(-i) * b**(i-j) * nchoosek(i, j) * Q_in[i]
        Q_out[j] = a**j*Q_out[j]
    return Q_out

# Tests:
np.testing.assert_array_equal(
    transQ(
        [np.array([[1, 1],[1, 1]]),
         np.array([[2, 2],[2, 2]]),
         np.array([[3, 3],[3, 3]])],
        0, 1),
    
    [np.array([[ 2.75,  2.75],
            [ 2.75,  2.75]]),
     np.array([[ 2.5,  2.5],
            [ 2.5,  2.5]]), 
     np.array([[ 0.75,  0.75],
            [ 0.75,  0.75]])])

In [ ]:
import math

def calcQSum(Q_in):
    from numpy import concatenate as con
    m = len(Q_in)
    n = Q_in[0].shape
    if m is 1:
        #([[2*Q_in[0],   np.zeros(n)], 
        #  [np.zeros(n), np.zeros(n)]])
        Q_sum = con((con((2*Q_in[0], np.zeros(n)), axis=1),
                     con((np.zeros(n), np.zeros(n)), axis=1)), axis=0)
    elif m is 2:
        # [[2*Q_in[0], Q_in[1]],
        #  [Q_in[1],   np.zeros(n)]]
        Q_sum = con((con((2*Q_in[0], Q_in[1]), axis=1),
                     con((Q_in[1], np.zeros(n)), axis=1)), axis=0)

    else:
        # [[2*Q_in[0], Q_in[1]],
        #  [Q_in[1], 2*Q_in[2]]]
        # m here is m+1 from Diss (starts at 0 there), so even and odd is interchanged
        Q_sum = con((con((2*Q_in[0], Q_in[1]), axis=1),
                     con((Q_in[1], 2*Q_in[2]), axis=1)), axis=0)
        
    for i1 in range(4, m+1, 2):
        if i1 is not m:
            #Q_sum = [[Q_sum], [[np.zeros((i1/2-1)*n, n)], [Q_in[i1]]], 
            #         [np.zeros(n, (i1/2-1)), Q_in[i1], 2*Q_in[i1+1]]]
            Q_sum = con((con((Q_sum, con((np.zeros((int(math.ceil(i1/2-1)*n[0]), n[1])), Q_in[i1-1]), axis=0)), axis=1),
                         con((np.zeros((n[0], int(math.ceil(i1/2-1)*n[1]))), Q_in[i1-1], 2*Q_in[i1]), axis=1)), axis=0)
        else:
            #Q_sum = [[Q_sum], [[np.zeros((i1/2-1)*n, n)], [Q_in[i1]]], 
            #         [np.zeros(n, (i1/2-1)), Q_in[i1], np.zeros(n)]]
            Q_sum = con((con((Q_sum, con((np.zeros((int(math.ceil(i1/2-1)*n[0]), n[1])), Q_in[i1-1]), axis=0)), axis=1),
                         con((np.zeros((n[0], int(math.ceil(i1/2-1)*n[1]))), Q_in[i1-1], np.zeros(n)), axis=1)), axis=0)
    Q_sum = 0.5*Q_sum
    return Q_sum

testarr = [np.array([[1, 1],
                     [1, 1]]),
           np.array([[2, 2],
                     [2, 2]]),
           np.array([[3, 3],
                     [3, 3]]),
           np.array([[4, 4],
                     [4, 4]]),
           np.array([[5, 5],
                     [5, 5]])]

# Tests:
np.testing.assert_array_equal(
    calcQSum(testarr[0:1]),
    np.array([[ 1.,  1.,  0.,  0.],
              [ 1.,  1.,  0.,  0.],
              [ 0.,  0.,  0.,  0.],
              [ 0.,  0.,  0.,  0.]]))

np.testing.assert_array_equal(
    calcQSum(testarr[0:2]),
    np.array([[ 1.,  1.,  1.,  1.],
              [ 1.,  1.,  1.,  1.],
              [ 1.,  1.,  0.,  0.],
              [ 1.,  1.,  0.,  0.]]))

np.testing.assert_array_equal(
    calcQSum(testarr[0:3]),
    np.array([[ 1.,  1.,  1.,  1.],
              [ 1.,  1.,  1.,  1.],
              [ 1.,  1.,  3.,  3.],
              [ 1.,  1.,  3.,  3.]]))

np.testing.assert_array_equal(
    calcQSum(testarr[0:4]),
    np.array([[ 1.,  1.,  1.,  1., 0, 0],
              [ 1.,  1.,  1.,  1., 0, 0],
              [ 1.,  1.,  3.,  3., 2, 2],
              [ 1.,  1.,  3.,  3., 2, 2],
              [ 0.,  0.,  2.,  2., 0, 0],
              [ 0.,  0.,  2.,  2., 0, 0]]))

np.testing.assert_array_equal(
    calcQSum(testarr[0:5]),
    np.array([[ 1.,  1.,  1.,  1., 0, 0],
              [ 1.,  1.,  1.,  1., 0, 0],
              [ 1.,  1.,  3.,  3., 2, 2],
              [ 1.,  1.,  3.,  3., 2, 2],
              [ 0.,  0.,  2.,  2., 5, 5],
              [ 0.,  0.,  2.,  2., 5, 5]]))


In [ ]:
################################################
# Demo Picos                                   #
# http://picos.zib.de/intro.html#first-example #
################################################

#generate data
A = [   cvx.sparse([[0 ,2 ,-1],
                    [-1,0 ,2 ],
                    [0 ,1 ,0 ]]),
        cvx.sparse([[1 ,2 ,0 ],
                    [2 ,0 ,0 ]]),
        cvx.sparse([[0 ,2 ,2 ]]),
    ]
K = cvx.sparse([[1 ,1 ,1 ],
                [1 ,-5,-5]])

#size of the data
s = len(A)
m = A[0].size[0]
l = [ Ai.size[1] for Ai in A ]
r = K.size[1]


#creates a problem and the optimization variables
prob = pic.Problem()
mu = prob.add_variable('mu',s)
print mu
Z  = [prob.add_variable('Z[' + str(i) + ']', (l[i],r))
      for i in range(s)]
print Z

#convert the constants into params of the problem
A = pic.new_param('A',A)
print A
K = pic.new_param('K',K)

#add the constraints
prob.add_constraint( pic.sum([ A[i]*Z[i] for i in range(s)], #summands
                            'i',                            #name of the index
                            '[s]'                           #set to which the index belongs
                           ) == K
                   )
prob.add_list_of_constraints( [ abs(Z[i]) < mu[i] for i in range(s)], #constraints
                              'i',                                    #index of the constraints
                              '[s]'                                   #set to which the index belongs
                            )

#sets the objective
prob.set_objective('min', 1 | mu ) # scalar product of the vector of all ones with mu

print '###############################################'
#display the problem
print prob

#call to the solver cvxopt
sol = prob.solve(solver='cvxopt', verbose = 0)

#show the value of the optimal variable
print '\n  mu ='
print mu

#show the dual variable of the equality constraint
print'\nThe optimal dual variable of the'
print prob.get_constraint(0)
print 'is :'
print prob.get_constraint(0).dual

In [ ]:
################
# Demo 2 Picos #
################

import numpy as s
def testAx_less_b():
    A = cvx.matrix(s.eye(3))
    c = cvx.matrix(s.ones( (3,1) ) )

    prob=pic.Problem()
    AA = pic.new_param('A', A)
    cc = pic.new_param('c', c)
    bb = pic.new_param('b', s.ones( (3,1) ))
    xx = prob.add_variable('x', 3)

    prob.set_objective('max', cc.T*xx)
    prob.add_constraint(AA*xx < bb)
    print prob

    #    prob.solve(verbose=1, solver='cvxopt')
    prob.solve()
    x = xx.value
    print x
testAx_less_b()